In [5]:
# Задание на повторение материала предыдущего семестра

In [6]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [7]:
# Генерируем уникальный seed
my_code = "Verchenko"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [4]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("datasets/russian_demography.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/russian_demography.csv'

In [375]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [376]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [377]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [378]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [379]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [380]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [381]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [382]:
sc_train

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,0.814815,0.653367,0.701299,0.392344,0.565074,0.066387,Altai Republic
1,0.037037,0.466334,0.34632,0.358852,0.712598,0.640828,Republic of Khakassia
2,0.296296,0.336658,0.181818,0.425837,0.488097,0.555716,Amur Oblast
3,0.555556,0.341646,0.207792,0.45933,0.128236,0.723714,Kamchatka Krai
4,0.740741,0.401496,0.38961,0.5311,0.41199,0.730261,Irkutsk Oblast
...,...,...,...,...,...,...,...
1384,0.888889,0.551122,0.406926,0.263158,0.463076,0.375409,Kabardino-Balkar Republic
1385,0.518519,0.067332,0.121212,0.875598,0.498523,0.772162,Ivanovo Oblast
1386,0.37037,0.244389,0.134199,0.550239,0.433015,0.674087,Krasnoyarsk Krai
1387,0.074074,0.241895,0.151515,0.574163,0.858732,0.433023,Tambov Oblast


In [383]:
# Вспоминаем алгоритмы решения задачи регрессии: линейную регрессию и метод k ближайших соседей
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

In [384]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]

In [385]:
# Обучаем модели
for model in r_models:
    model.fit(x_train, y_train)

In [386]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.02005965686156148
0.029364218382941364
0.01996402213996237
0.029364218382941364
0.011701859181107789
0.01220381740257272
0.013005800599327084


In [387]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [388]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.011435039776008434


In [389]:
# Вспоминаем алгоритмы решения задачи классификации:
# логистическую регрессию, наивный байесовский классификатор и (снова) метод k ближайших соседей
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga'))
c_models.append(LogisticRegression(penalty='l1', solver='saga'))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

In [390]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [391]:
# Обучаем модели
for model in c_models:
    model.fit(x_train, y_train)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:509: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [392]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.26330357718811326
0.11074821855346845
0.03947162600875228
0.04636753051553553
0.03947162600875228
0.030145823176017668
0.029288654957541075
0.029819572090476133
0.3635663215784113
0.2486164456002101
0.21982707117696315


In [393]:
# Выбираем лучшую модель
i_min = f1s.index(min(f1s))
best_c_model = c_models[i_min]
best_c_model

MultinomialNB(alpha=0.5)

In [394]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.030228467899282484


In [395]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [396]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.814815,0.653367,0.701299,0.392344,0.565074,0.066387
1,0.037037,0.466334,0.34632,0.358852,0.712598,0.640828
2,0.296296,0.336658,0.181818,0.425837,0.488097,0.555716
3,0.555556,0.341646,0.207792,0.45933,0.128236,0.723714
4,0.740741,0.401496,0.38961,0.5311,0.41199,0.730261
...,...,...,...,...,...,...
458,0.407407,0.264339,0.164502,0.545455,0.541268,0.44363
459,1.0,0.21197,0.177489,0.660287,0.861512,0.617651
460,0.925926,0.42394,0.393939,0.492823,0.615465,0.723714
461,0.444444,0.269327,0.233766,0.61244,0.358297,0.553751


In [397]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [398]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.29466397435883146
0.2545939616984991
0.24067614849687458
0.23033961045590134


In [399]:
# Выбираем лучшую модель
i_min = sils.index(min(sils))
best_k_model = k_models[i_min]
print(best_k_model)
print(sils[i_min])



KMeans(n_clusters=50)
0.23033961045590134


In [400]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25)
]

In [401]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[KNeighborsRegressor(n_neighbors=25), Lasso(), ElasticNet(), ElasticNet(l1_ratio=0.25)]


In [402]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("datasets/weather.csv")
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,"468,0","0,0","24,9","31,0","100,0","4,0","0,0","993,8","4,0","-3,0","-0,3","-0,4"
1,"140,0","0,0","27,9","22,0","20,0","1,0","0,0","1004,1","4,0","-2,0","0,6","-1,0"
2,"464,0","0,0","25,6","28,0","20,0","2,0","1,0","1001,2","4,0","-7,0","-1,8","-0,3"
3,"467,0","0,0","26,7","26,0","100,0","3,0","0,0","992,3","4,0","-1,0","1,2","-1,3"
4,"138,0","0,0","29,1","22,0","20,0","1,0","0,0","1001,3","4,0","-2,0","2,5","-1,3"
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,"474,0","0,1","6,4","95,0","4,0","1,0","3,0","999,9","4,0","-1,0","5,9","1,5"
1435,"65,0","0,0","6,4","93,0","10,0","4,0","3,0","983,9","4,0","-1,0","5,2","3,1"
1436,"159,0","0,1","8,5","83,0","10,0","2,0","1,0","969,7","4,0","1,0","1,4","1,9"
1437,"99,0","0,1","7,5","91,0","10,0","3,0","3,0","995,4","4,0","-4,0","2,7","1,7"


In [403]:
data['water_level'] = data['water_level'].str.replace(',','.')
data['precipitation'] = data['precipitation'].str.replace(',','.')
data['temperature'] = data['temperature'].str.replace(',','.')
data['humidity'] = data['humidity'].str.replace(',','.')
data['visibility'] = data['visibility'].str.replace(',','.')
data['wind'] = data['wind'].str.replace(',','.')
data['weather'] = data['weather'].str.replace(',','.')
data['pressure'] = data['pressure'].str.replace(',','.')
data['fire'] = data['fire'].str.replace(',','.')                       
data['wl_changer'] = data['wl_change'].str.replace(',','.')                       
data['temp_change'] = data['temp_change'].str.replace(',','.')                      
data['pressure_change'] = data['pressure_change'].str.replace(',','.')                         

In [404]:
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change,wl_changer
0,468.0,0.0,24.9,31.0,100.0,4.0,0.0,993.8,4.0,"-3,0",-0.3,-0.4,-3.0
1,140.0,0.0,27.9,22.0,20.0,1.0,0.0,1004.1,4.0,"-2,0",0.6,-1.0,-2.0
2,464.0,0.0,25.6,28.0,20.0,2.0,1.0,1001.2,4.0,"-7,0",-1.8,-0.3,-7.0
3,467.0,0.0,26.7,26.0,100.0,3.0,0.0,992.3,4.0,"-1,0",1.2,-1.3,-1.0
4,138.0,0.0,29.1,22.0,20.0,1.0,0.0,1001.3,4.0,"-2,0",2.5,-1.3,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,474.0,0.1,6.4,95.0,4.0,1.0,3.0,999.9,4.0,"-1,0",5.9,1.5,-1.0
1435,65.0,0.0,6.4,93.0,10.0,4.0,3.0,983.9,4.0,"-1,0",5.2,3.1,-1.0
1436,159.0,0.1,8.5,83.0,10.0,2.0,1.0,969.7,4.0,"1,0",1.4,1.9,1.0
1437,99.0,0.1,7.5,91.0,10.0,3.0,3.0,995.4,4.0,"-4,0",2.7,1.7,-4.0


In [405]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = ['water_level']
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['wind', 'humidity', 'pressure_change', 'weather', 'temperature']


In [406]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.


In [407]:
all_lb =  x_labels + y_label

In [408]:

data=data[list(all_lb)]
data

,wind,humidity,pressure_change,weather,temperature,water_level
0,4.0,31.0,-0.4,0.0,24.9,468.0
1,1.0,22.0,-1.0,0.0,27.9,140.0
2,2.0,28.0,-0.3,1.0,25.6,464.0
3,3.0,26.0,-1.3,0.0,26.7,467.0
4,1.0,22.0,-1.3,0.0,29.1,138.0
...,...,...,...,...,...,...
1434,1.0,95.0,1.5,3.0,6.4,474.0
1435,4.0,93.0,3.1,3.0,6.4,65.0
1436,2.0,83.0,1.9,1.0,8.5,159.0
1437,3.0,91.0,1.7,3.0,7.5,99.0


In [409]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

288


In [410]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

863 288 288


In [411]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = all_lb

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['wind', 'humidity', 'pressure_change',
                                  'weather', 'temperature', 'water_level'])])

In [412]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [413]:
# Устанавливаем названия столбцов
column_names = all_lb
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [414]:
sc_train

,wind,humidity,pressure_change,weather,temperature,water_level
0,0.2,0.758242,0.208791,0.125,0.558087,0.510008
1,0.3,0.571429,0.351648,0.000,0.740319,0.252202
2,0.1,0.604396,0.538462,0.125,0.692483,0.373899
3,0.0,0.769231,0.307692,0.125,0.596811,0.244195
4,0.2,0.483516,0.296703,0.000,0.760820,0.546837
...,...,...,...,...,...,...
858,0.1,0.362637,0.263736,0.250,0.776765,0.357886
859,0.1,0.747253,0.318681,0.125,0.407745,0.245797
860,0.0,0.945055,0.395604,0.375,0.517084,0.501201
861,0.0,0.824176,0.428571,0.000,0.134396,0.248999


In [415]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [416]:
# Обучаем модели
for model in my_models1:
    model.fit(x_train, y_train)

In [417]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in my_models1:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.030796046829457153
0.03338988118463235
0.03338988118463235
0.03338988118463235


In [418]:
# Выбираем лучшую модель # Укажите, какая модель решает задачу лучше других.
i_min = mses.index(min(mses))
best_my_model = my_models1[i_min]
print('Модель ', best_my_model, 'решает задачу лучше других.') 

Модель  KNeighborsRegressor(n_neighbors=25) решает задачу лучше других.


In [419]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.09457730151360064


In [420]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25)
]

In [421]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[LogisticRegression(), LogisticRegression(penalty='l1', solver='saga'), KNeighborsClassifier(n_neighbors=15), MultinomialNB(alpha=0.5), LogisticRegression(l1_ratio=0.75, penalty='elasticnet', solver='saga')]


In [422]:
# Загрузим данные для задачи классификации
data = pd.read_csv("datasets/zoo2.csv")
data

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [423]:
#data.drop(['animal_name'], axis='columns', inplace=True)

In [424]:
#data.head()

In [425]:
data['animal_name'] = pd.get_dummies(data['animal_name'])
data.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,0,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,0,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,0,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,0,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,0,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3


In [426]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(data.columns)
n_x = 8

y_label = ['class_type']
x_labels = random.sample(columns[:-1], n_x)
all_lb1 =  x_labels + y_label
print(x_labels)

['predator', 'hair', 'breathes', 'aquatic', 'feathers', 'fins', 'airborne', 'milk']


In [427]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

In [428]:
data=data[list(all_lb1)]
data.head()

,predator,hair,breathes,aquatic,feathers,fins,airborne,milk,class_type
0,0,0,1,1,0,0,0,0,3
1,0,0,1,0,0,0,0,0,3
2,1,0,1,0,0,0,0,0,3
3,1,0,1,0,0,0,0,0,3
4,0,0,1,0,0,0,0,0,3


In [429]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

9


In [430]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

25 9 9


In [431]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = x_labels#all_lb1

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['predator', 'hair', 'breathes', 'aquatic',
                                  'feathers', 'fins', 'airborne', 'milk'])])

In [432]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [433]:
# Устанавливаем названия столбцов
column_names = all_lb1
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [434]:
sc_train

,predator,hair,breathes,aquatic,feathers,fins,airborne,milk,class_type
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,6.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,6.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,6.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0
5,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,6.0
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
8,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0
9,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,5.0


In [435]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_label])
y_test = np.ravel(sc_test[y_label])
y_val = np.ravel(sc_val[y_label])

In [436]:
# Обучаем модели
for model2 in my_models2:
    model2.fit(x_train, y_train)

In [438]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in my_models2:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.41111111111111115
0.4074074074074074
0.2735042735042735
0.4074074074074074
0.4074074074074074


In [439]:
# Выбираем лучшую модель # Укажите, какая модель решает задачу лучше других.
i_min = f1s.index(min(f1s))
best_my_models2 = my_models2[i_min]
print('Модель ', best_my_models2, 'решает задачу лучше других.') 

Модель  KNeighborsClassifier(n_neighbors=15) решает задачу лучше других.


In [440]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_my_models2.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.17777777777777778
